In [1]:
print("cosine trunc weight: {}".format(ch.nn.functional.cosine_similarity(trunc_multi_log_reg.weight, ground_truth.weight)))
print("cosine trunc bias: {}".format(ch.nn.functional.cosine_similarity(trunc_multi_log_reg.bias.unsqueeze(0), ground_truth.bias.unsqueeze(0))))

NameError: name 'ch' is not defined

In [ ]:
print("cosine standard weight: {}".format(ch.nn.functional.cosine_similarity(standard_multi_log_reg.weight, ground_truth.weight)))
print("cosine standard bias: {}".format(ch.nn.functional.cosine_similarity(standard_multi_log_reg.bias.unsqueeze(0), ground_truth.bias.unsqueeze(0))))

In [ ]:
print("cosine gumbel ce weight: {}".format(ch.nn.functional.cosine_similarity(gumbel_ce_multi_log_reg.weight, ground_truth.weight)))
print("cosine gumbel ce bias: {}".format(ch.nn.functional.cosine_similarity(gumbel_ce_multi_log_reg.bias.unsqueeze(0), ground_truth.bias.unsqueeze(0))))

In [ ]:
from torch.distributions.transforms import SigmoidTransform
from torch.distributions.transformed_distribution import TransformedDistribution

gumbel = Gumbel(0, 1)

# logistic distribution
base_distribution = Uniform(0, 1)
transforms_ = [SigmoidTransform().inv]
logistic = TransformedDistribution(base_distribution, transforms_)

phi = Identity()


m = ch.nn.Linear(10, 2)
x = ch.rand(1000, 10)
w = ch.randn(10, 2)
y = (x @ w + gumbel.sample([1000, 2])).argmax(1)

m_ = ch.nn.Linear(10, 1)
x_ = ch.rand(1000, 10)
w_ = ch.randn(10, 1)
y_ = (x_ @ w_ + logistic.sample([1000, 1])).argmax(1).float()

out = m(x)
loss_ = GumbelCE.apply(out, y)
g, = ch.autograd.grad(loss_, [out])

gt_loss = nn.CrossEntropyLoss()(out, y)
gt_g, = ch.autograd.grad(gt_loss, [out])

trunc_loss = trunc_ce(out, y)
gt_trunc, = ch.autograd.grad(trunc_loss, [out])

out_ = m_(x_)
trunc_bce_loss = trunc_bce(out_, y_.unsqueeze(1))
g_bce, = ch.autograd.grad(trunc_bce_loss, [out_])

print('Correlation between real and estimated gradient: ',
        (gt_g * g).sum() / (gt_g.norm() * g.norm()))

print('Correlation between real and truncated gradient: ',
        (gt_g * gt_trunc).sum() / (gt_g.norm() * gt_trunc.norm()))

print('Correlation between trunc bce and real ce gradient: ', 
     (g_bce * gt_g).sum() / (g_bce.norm() * gt_g.norm()))